In [ ]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.text import one_hot
## word embeddingb representation
from tensorflow.keras.datasets import imdb

from tensorflow.keras.layers import Embedding, SimpleRNN, Dense
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.utils import pad_sequences
from tensorflow.keras.models import Sequential

In [ ]:
## mapping of word index back to words
from tensorflow.keras.models import load_model

word_index=imdb.get_word_index()
word_index
reverse_word_index={value: key for key , value in word_index.items()}

In [ ]:
model=load_model('simple_rnn_imdb.h5')
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_2 (Embedding)         │ (32, 500, 128)         │     1,280,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn_2 (SimpleRNN)        │ (32, 128)              │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (32, 1)                │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,313,027 (5.01 MB)

 Trainable params: 1,313,025 (5.01 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)

In [ ]:
model.get_weights()

[array([[-1.6411581e+00,  4.6768847e-01, -5.2099699e-01, ...,
          5.0694406e-01, -7.0770746e-01, -1.3767986e-01],
        [ 4.7107585e-02,  8.0239028e-02, -4.5777142e-02, ...,
          4.1862302e-02, -5.4529525e-02,  1.9620337e-02],
        [ 2.0234793e-01,  1.6619286e-01, -5.0234236e-02, ...,
         -1.8476576e-01, -7.2453017e-03, -5.7075076e-02],
        ...,
        [-6.6145219e-02,  7.1700863e-03,  8.7726586e-02, ...,
         -1.1145132e-01, -2.2126671e-02, -1.0864381e-01],
        [-1.7871570e-02,  3.5337415e-03,  4.6453740e-02, ...,
         -8.2023960e-04, -6.9262654e-02,  5.2732389e-02],
        [-7.3502220e-02, -1.4140312e-02, -6.4777059e-04, ...,
          2.5901150e-02,  9.1684116e-03, -1.1793770e-02]], dtype=float32),
 array([[ 0.05621806, -0.09313759, -0.01058604, ...,  0.03114016,
         -0.09278074, -0.15135163],
        [-0.04376391, -0.08900852,  0.14224932, ...,  0.07101905,
         -0.04220556, -0.1113923 ],
        [-0.0730579 , -0.02220833, -0.10751775

In [ ]:
def decode_review(encoded_review):
    return ' '.join([reverse_word_index.get(i-3,'?')for i in encoded_review])


def preprocess_text(text):
    words=text.lower().split()
    encoded_review=[word_index.get(word,2)+3 for word in words]
    padded_review=sequence.pad_sequences([encoded_review], maxlen=50)
    return padded_review 

In [ ]:
## prediction function
def predict_sentiment(review):
    preproces_review=preprocess_text(review)
    prediction=model.predict(preproces_review)
    sentiment='Positive' if prediction[0][0]>0.5 else 'Negative'
    return sentiment, prediction[0][0]

In [ ]:
#step=4 user Input and prediction

example_review="this movie was fantastic! the acting was great and the plot was thrilling."
sentiment, score=predict_sentiment(example_review)
print(f'Review: {example_review}')
print(f'Sentiment: {sentiment}')
print(f'Prediction score: {score}')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
Review: this movie was fantastic! the acting was great and the plot was thrilling.
Sentiment: Positive
Prediction score: 0.8549903035163879


In [ ]:

example_review="this movie was worst! the acting was bad."
sentiment, score=predict_sentiment(example_review)
print(f'Review: {example_review}')
print(f'Sentiment: {sentiment}')
print(f'Prediction score: {score}')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
Review: this movie was worst! the acting was bad.
Sentiment: Positive
Prediction score: 0.7822667956352234


In [ ]:
import tensorflow as tf
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense

# Parameters
voc_size = 10000   # Vocabulary size
max_len = 100      # Maximum length of a review
embedding_dim = 32 # Embedding dimension

# Load IMDb Dataset
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=voc_size)

# Pad Sequences
X_train = pad_sequences(X_train, maxlen=max_len, padding='post')
X_test = pad_sequences(X_test, maxlen=max_len, padding='post')

# Model Architecture
model = Sequential()
model.add(Embedding(input_dim=voc_size, output_dim=embedding_dim, input_length=max_len))
model.add(SimpleRNN(64, activation='tanh', return_sequences=False))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

# Training
history = model.fit(X_train, y_train, epochs=5, batch_size=64, validation_data=(X_test, y_test))

# Evaluation
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f'Test Accuracy: {test_acc}')

# Prediction Function
word_index = imdb.get_word_index()
reverse_word_index = {value: key for (key, value) in word_index.items()}

def decode_review(encoded_review):
    return ' '.join([reverse_word_index.get(i - 3, '?') for i in encoded_review])

def preprocess_review(review):
    review = review.lower()
    words = review.split()
    encoded_review = [word_index.get(word, 2) for word in words]  # 2 is for unknown words
    padded_review = pad_sequences([encoded_review], maxlen=max_len, padding='post')
    return padded_review

def predict_sentiment(review):
    processed_review = preprocess_review(review)
    score = model.predict(processed_review)[0][0]
    sentiment = 'Positive' if score > 0.5 else 'Negative'
    return sentiment, score

# Example Prediction
example_review = "This movie was the worst! The acting was disgusting."
sentiment, score = predict_sentiment(example_review)
print(f'Review: {example_review}')
print(f'Sentiment: {sentiment}')
print(f'Prediction score: {score}')


d:\python\deep learning\venv\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn (SimpleRNN)          │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

Epoch 1/5
391/391 ━━━━━━━━━━━━━━━━━━━━ 13s 26ms/step - accuracy: 0.5994 - loss: 0.6518 - val_accuracy: 0.8022 - val_loss: 0.4486
Epoch 2/5
391/391 ━━━━━━━━━━━━━━━━━━━━ 9s 24ms/step - accuracy: 0.8198 - loss: 0.4202 - val_accuracy: 0.8114 - val_loss: 0.4356
Epoch 3/5
391/391 ━━━━━━━━━━━━━━━━━━━━ 11s 24ms/step - accuracy: 0.8559 - loss: 0.3575 - val_accuracy: 0.8394 - val_loss: 0.3818
Epoch 4/5
391/391 ━━━━━━━━━━━━━━━━━━━━ 10s 24ms/step - accuracy: 0.8988 - loss: 0.2672 - val_accuracy: 0.8282 - val_loss: 0.4238
Epoch 5/5
391/391 ━━━━━━━━━━━━━━━━━━━━ 9s 24ms/step - accuracy: 0.9190 - loss: 0.2268 - val_accuracy: 0.8172 - val_loss: 0.4586
782/782 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - accuracy: 0.8157 - loss: 0.4632
Test Accuracy: 0.8171600103378296
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 277ms/step
Review: This movie was the worst! The acting was disgusting.
Sentiment: Positive
Prediction score: 0.9412443041801453


In [ ]:
import numpy as np

print(f'Positive reviews: {np.sum(y_train)}')
print(f'Negative reviews: {len(y_train) - np.sum(y_train)}')


Positive reviews: 12500
Negative reviews: 12500


In [ ]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Accuracy: {accuracy}')
print(f'Test Loss: {loss}')


782/782 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.8157 - loss: 0.4632
Test Accuracy: 0.8171600103378296
Test Loss: 0.4585970640182495


In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

y_pred = model.predict(X_test)
y_pred_classes = (y_pred > 0.5).astype("int32")
conf_matrix = confusion_matrix(y_test, y_pred_classes)
print(conf_matrix)
print(classification_report(y_test, y_pred_classes))


782/782 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step
[[10053  2447]
 [ 2124 10376]]
              precision    recall  f1-score   support

           0       0.83      0.80      0.81     12500
           1       0.81      0.83      0.82     12500

    accuracy                           0.82     25000
   macro avg       0.82      0.82      0.82     25000
weighted avg       0.82      0.82      0.82     25000

